# Running Secondary Index Query with Regex on String Key Stored in Aerospike

### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [21]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [22]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [23]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

####  Java Client Imports

In [24]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

// Expression Filter imports
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.Expression;
import com.aerospike.client.query.RegexFlag;

//Query Imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.RecordSet;

In [25]:
%sh aql -c "set key_send true"

In [26]:
%sh aql -f "../aqlScripts/insert.aql"

In [27]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");
for(int i=0; i<10; i++){
    Key key = new Key("test", "testset", "key"+i);
    System.out.println("key"+i+ " : "+client.get(null, key));
}

Initialized the client and connected to the cluster.
key0 : (gen:1),(exp:453831888),(bins:(name:Sandra),(age:34))
key1 : (gen:1),(exp:453831888),(bins:(name:Jack),(age:26))
key2 : (gen:1),(exp:453831888),(bins:(name:Jill),(age:20))
key3 : (gen:1),(exp:453831888),(bins:(name:James),(age:38))
key4 : (gen:1),(exp:453831888),(bins:(name:Jim),(age:46))
key5 : (gen:1),(exp:453831888),(bins:(name:Julia),(age:62))
key6 : (gen:1),(exp:453831888),(bins:(name:Sally),(age:32))
key7 : (gen:1),(exp:453831888),(bins:(name:Sean),(age:24))
key8 : (gen:1),(exp:453831888),(bins:(name:Sam),(age:12))
key9 : (gen:1),(exp:453831888),(bins:(name:Susan),(age:42))


In [29]:
//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");

//Filter key by expressions
Expression recKeyExp = Exp.build( 
    Exp.regexCompare(".*[2,3]", RegexFlag.ICASE|RegexFlag.NEWLINE, Exp.key(Exp.Type.STRING))
    ); 
QueryPolicy qp = new QueryPolicy();
qp.filterExp = recKeyExp;
RecordSet rs = client.query(qp, stmt);

while(rs.next()){
    Record r = rs.getRecord();
    Key thisKey = rs.getKey();  
    System.out.println(thisKey);
    System.out.println(r);
}

test:testset:key2:c10b2c764e604c439039804ec947fa2b18520d1a
(gen:1),(exp:453831888),(bins:(name:Jill),(age:20))
test:testset:key3:10fd8f59adf1833152e439a2e03c19efcb12c145
(gen:1),(exp:453831888),(bins:(name:James),(age:38))


# Cleanup

In [5]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"